# understanding_hyena.ipynb

In [2]:
from shap_analysis.ism_utils import *
multitasking_path1 = '/data/leslie/sarthak/hyena/hyena-dna/outputs/2024-03-27/18-39-11-031863/checkpoints/25-val_loss=0.52186.ckpt' #the 25 epoch one
mult = ISMUtils('DNase_allcelltypes', multitasking_path1, classification=True)

In [4]:
# mult.model
mult.backbone

DNAEmbeddingModel(
  (backbone): LMBackbone(
    (embeddings): GPT2Embeddings(
      (word_embeddings): Embedding(16, 128)
    )
    (layers): ModuleList(
      (0): Block(
        (mixer): HyenaOperator(
          (activation): Identity()
          (dropout): Dropout(p=0.0, inplace=False)
          (out_proj): Linear(in_features=128, out_features=128, bias=True)
          (in_proj): Linear(in_features=128, out_features=384, bias=True)
          (short_filter): Conv1d(384, 384, kernel_size=(3,), stride=(1,), padding=(2,), groups=384)
          (filter_fn): HyenaFilter(
            (dropout): Dropout(p=0.0, inplace=False)
            (pos_emb): PositionalEmbedding()
            (implicit_filter): Sequential(
              (0): Linear(in_features=5, out_features=64, bias=True)
              (1): Sin()
              (2): Linear(in_features=64, out_features=64, bias=True)
              (3): Sin()
              (4): Linear(in_features=64, out_features=64, bias=True)
              (5): Sin()

In [12]:
mult.backbone.backbone

LMBackbone(
  (embeddings): GPT2Embeddings(
    (word_embeddings): Embedding(16, 128)
  )
  (layers): ModuleList(
    (0): Block(
      (mixer): HyenaOperator(
        (activation): Identity()
        (dropout): Dropout(p=0.0, inplace=False)
        (out_proj): Linear(in_features=128, out_features=128, bias=True)
        (in_proj): Linear(in_features=128, out_features=384, bias=True)
        (short_filter): Conv1d(384, 384, kernel_size=(3,), stride=(1,), padding=(2,), groups=384)
        (filter_fn): HyenaFilter(
          (dropout): Dropout(p=0.0, inplace=False)
          (pos_emb): PositionalEmbedding()
          (implicit_filter): Sequential(
            (0): Linear(in_features=5, out_features=64, bias=True)
            (1): Sin()
            (2): Linear(in_features=64, out_features=64, bias=True)
            (3): Sin()
            (4): Linear(in_features=64, out_features=64, bias=True)
            (5): Sin()
            (6): Linear(in_features=64, out_features=128, bias=False)
    

In [10]:
#we can see the model's makeup, let's look at the backbone first
data = mult.dataset[0][0].unsqueeze(0)
print(data.shape, data)

torch.Size([1, 1023]) tensor([[ 9,  9, 10,  ...,  8, 10,  8]])


In [11]:
data1 = mult.backbone.backbone(data)
print(data1.shape, data1) #this is the output of the backbone? without the Lm head

torch.Size([1, 1023, 128]) tensor([[[-0.2054,  0.3849, -0.6309,  ..., -0.2791, -0.2442,  0.3236],
         [-0.1926,  0.3297, -0.6444,  ..., -0.2739, -0.2438,  0.2496],
         [-0.1883,  0.2711, -0.7615,  ..., -0.2748, -0.2304,  0.3415],
         ...,
         [-0.1865, -1.4684, -0.0480,  ..., -0.2488, -0.2046,  0.4700],
         [-0.1858, -1.8168, -0.2808,  ..., -0.2550, -0.1811,  0.4815],
         [-0.1853, -1.4642, -0.5938,  ..., -0.2672, -0.1692, -0.1404]]],
       grad_fn=<NativeLayerNormBackward0>)


In [19]:
data2 = mult.backbone(data)
print(len(data2), data2[0].shape,data2[1]) #this is the output of the backbone with the Lm head
#here we see it's again 128 for each of the sequence elements
#the none that is the second output of the backbone is something
print(data2)

2 torch.Size([1, 1023, 128]) None
(tensor([[[-0.2054,  0.3849, -0.6309,  ..., -0.2791, -0.2442,  0.3236],
         [-0.1926,  0.3297, -0.6444,  ..., -0.2739, -0.2438,  0.2496],
         [-0.1883,  0.2711, -0.7615,  ..., -0.2748, -0.2304,  0.3415],
         ...,
         [-0.1865, -1.4684, -0.0480,  ..., -0.2488, -0.2046,  0.4700],
         [-0.1858, -1.8168, -0.2808,  ..., -0.2550, -0.1811,  0.4815],
         [-0.1853, -1.4642, -0.5938,  ..., -0.2672, -0.1692, -0.1404]]],
       grad_fn=<NativeLayerNormBackward0>), None)


In [20]:
#now let's go through the decoder head
mult.decoder #this is the multitasking, goes from 128 to 322

#ahhhh it compresses it using pool

SequenceDecoder(
  (output_transform): Linear(in_features=128, out_features=322, bias=True)
)

In [22]:
#this is the output of the decoder
data3 = mult.decoder(data2[0])
print(data3.shape, data3)

torch.Size([1, 322]) tensor([[-0.5049, -0.3075,  1.8645,  3.5166,  2.3568,  2.1814,  3.6842,  2.7295,
          1.4710,  3.9252,  3.4050,  0.4614, -0.1982,  3.5546,  2.3553,  0.5559,
          0.3023,  1.0751,  0.1795,  3.8263,  2.1100,  1.1665,  1.9441,  1.7290,
          0.9602, -0.3431,  3.2601,  0.0533,  1.0779, -1.2215,  3.5453,  0.3740,
          2.3839, -1.4882,  2.3575,  1.9609,  2.0365,  3.3059, -0.0380, -0.1618,
         -0.4768, -1.1979, -0.3500, -0.5653, -1.2556,  0.7909,  0.9702, -0.0731,
         -0.7344, -0.1408, -0.5081,  0.8514,  0.1175,  2.1414, -0.1052,  0.5142,
          2.5591,  3.6029,  0.8347, -0.0326,  3.7486,  3.3513,  0.5822,  3.4517,
          0.7170,  4.8885,  4.4089,  2.3378, -0.0816, -0.5852,  0.1949,  4.2735,
         -0.2658,  1.2450,  4.0940,  4.2701,  4.4384,  4.4356,  0.2521,  0.0809,
         -0.5427,  4.5116,  4.6212,  1.4247,  2.5012,  3.3803,  1.9500,  2.2894,
          2.1764,  1.4053,  0.6714,  1.6953,  2.1111,  0.3320,  4.6367,  0.2180,
       

In [27]:
restrict = lambda x: (
    torch.cumsum(x, dim=-2)
    / torch.arange(
        1, 1 + x.size(-2), device=x.device, dtype=x.dtype
    ).unsqueeze(-1)
)[..., -l_output:, :]
l_output = 1024
out = restrict(data2[0])
print(out.shape)
#hmmm so it does indeed keep the shape...

torch.Size([1, 1023, 128])


In [26]:
restrict = lambda x: torch.cumsum(x, dim=-2)[..., -l_output:, :]
out2 = restrict(data2[0])
print(out2.shape)
#ok this also doesn't actually reduce the dimensionality, so what does??

torch.Size([1, 1023, 128])


In [28]:
#this is mode last
restrict = lambda x: x[..., -l_output:, :] #yeah obviously this doesn't reduce the shape at all!!!
out3 = restrict(data2[0])
print(out3.shape)

torch.Size([1, 1023, 128])


In [30]:
#I think l_output is actually 0, if that is the case, then what we have is
l_output = 0
restrict = lambda x: (
    torch.cumsum(x, dim=-2)
    / torch.arange(
        1, 1 + x.size(-2), device=x.device, dtype=x.dtype
    ).unsqueeze(-1)
)[..., -l_output:, :]
# l_output = 1024
out = restrict(data2[0])
print(out.shape)

torch.Size([1, 1023, 128])


In [33]:
out[:,-0,:] == out[:,-1,:]

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, F

In [35]:
out[:,0,:] == out[:,-0,:] #yeah so -0 and 0 are identical...

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True]])

In [37]:
#wait so are we maybe messing up? in ism utils we define l_output=0 because that is what we see...
#maybe we're doing it wrong and that's not what the real decoder is doing? I'm so confused!
#if l_output is 1 then it takes -1:, but 0: means all, so like taking the full length!
l_output = 0
restrict = lambda x: (
    torch.cumsum(x, dim=-2)
    / torch.arange(
        1, 1 + x.size(-2), device=x.device, dtype=x.dtype
    ).unsqueeze(-1)
)[..., -l_output:, :]
# l_output = 1024
out = restrict(data2[0])
print(out.shape)

torch.Size([1, 1023, 128])


In [41]:
mult.decoder

SequenceDecoder(
  (output_transform): Linear(in_features=128, out_features=322, bias=True)
)

In [ ]:
#Oh my god, it turns 0 to 1, so we only take the last element
#the last element of pool is the same as just the average... oh my god!
#such a complex approach to do the most simple thing!!!

# figure out how the masking technique works when we are doing a language modeling task!!

In [ ]:
#let's figure this out!